In [1]:
import flask

ModuleNotFoundError: No module named 'flask'

In [3]:
!pip install flask flask-restplus

    100% |████████████████████████████████| 1.0MB 796kB/s ta 0:00:01


In [4]:
import flask

In [1]:
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'
app.run(host="0.0.0.0", port=5000)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.1 - - [01/Mar/2018 05:51:08] "GET / HTTP/1.1" 200 -


In [4]:
from flask import Flask
from flask_restplus import Api, Resource, fields
from werkzeug.contrib.fixers import ProxyFix

app = Flask(__name__)
app.wsgi_app = ProxyFix(app.wsgi_app)
api = Api(app, version='1.0', title='TodoMVC API',
    description='A simple TodoMVC API',
)

ns = api.namespace('todos', description='TODO operations')

todo = api.model('Todo', {
    'id': fields.Integer(readOnly=True, description='The task unique identifier'),
    'task': fields.String(required=True, description='The task details')
})


class TodoDAO(object):
    def __init__(self):
        self.counter = 0
        self.todos = []

    def get(self, id):
        for todo in self.todos:
            if todo['id'] == id:
                return todo
        api.abort(404, "Todo {} doesn't exist".format(id))

    def create(self, data):
        todo = data
        todo['id'] = self.counter = self.counter + 1
        self.todos.append(todo)
        return todo

    def update(self, id, data):
        todo = self.get(id)
        todo.update(data)
        return todo

    def delete(self, id):
        todo = self.get(id)
        self.todos.remove(todo)


DAO = TodoDAO()
DAO.create({'task': 'Build an API'})
DAO.create({'task': '?????'})
DAO.create({'task': 'profit!'})


@ns.route('/')
class TodoList(Resource):
    '''Shows a list of all todos, and lets you POST to add new tasks'''
    @ns.doc('list_todos')
    @ns.marshal_list_with(todo)
    def get(self):
        '''List all tasks'''
        return DAO.todos

    @ns.doc('create_todo')
    @ns.expect(todo)
    @ns.marshal_with(todo, code=201)
    def post(self):
        '''Create a new task'''
        return DAO.create(api.payload), 201


@ns.route('/<int:id>')
@ns.response(404, 'Todo not found')
@ns.param('id', 'The task identifier')
class Todo(Resource):
    '''Show a single todo item and lets you delete them'''
    @ns.doc('get_todo')
    @ns.marshal_with(todo)
    def get(self, id):
        '''Fetch a given resource'''
        return DAO.get(id)

    @ns.doc('delete_todo')
    @ns.response(204, 'Todo deleted')
    def delete(self, id):
        '''Delete a task given its identifier'''
        DAO.delete(id)
        return '', 204

    @ns.expect(todo)
    @ns.marshal_with(todo)
    def put(self, id):
        '''Update a task given its identifier'''
        return DAO.update(id, api.payload)


if __name__ == '__main__':
    app.run(host="0.0.0.0", port=5000)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.1 - - [01/Mar/2018 05:53:52] "GET / HTTP/1.1" 200 -
172.17.0.1 - - [01/Mar/2018 05:53:52] "GET /swaggerui/bower/swagger-ui/dist/css/typography.css HTTP/1.1" 200 -
172.17.0.1 - - [01/Mar/2018 05:53:52] "GET /swaggerui/bower/swagger-ui/dist/css/reset.css HTTP/1.1" 200 -
172.17.0.1 - - [01/Mar/2018 05:53:52] "GET /swaggerui/bower/swagger-ui/dist/css/screen.css HTTP/1.1" 200 -
172.17.0.1 - - [01/Mar/2018 05:53:52] "GET /swaggerui/bower/swagger-ui/dist/lib/object-assign-pollyfill.js HTTP/1.1" 200 -
172.17.0.1 - - [01/Mar/2018 05:53:52] "GET /swaggerui/bower/swagger-ui/dist/lib/jquery-1.8.0.min.js HTTP/1.1" 200 -
172.17.0.1 - - [01/Mar/2018 05:53:52] "GET /swaggerui/bower/swagger-ui/dist/lib/jquery.slideto.min.js HTTP/1.1" 200 -
172.17.0.1 - - [01/Mar/2018 05:53:52] "GET /swaggerui/bower/swagger-ui/dist/lib/jquery.wiggle.min.js HTTP/1.1" 200 -
172.17.0.1 - - [01/Mar/2018 05:53:52] "GET /swaggerui/bower/swagger-ui/dist/lib/jqu